### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170911

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M09D13', 'UTC 23:37')


# Settings

In [2]:
MAXTASKS = 200
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,error_message,id,output_url,queuetime(min),runtime(min),start_timestamp_ms,state,task_type,update_timestamp_ms
0,0.011200,1505345695705,reducedmeanrunoff_year_Y1960Y2014_V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,TNFZYMZ2BTRBXGNN7YMAA6BL,NaN,1.643183,1.654383,1.505346e+12,FAILED,EXPORT_IMAGE,1505345794968
1,0.009033,1505345695167,reducedmeanrunoff_month_Y1960Y2014M12V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,M5JDAGMMNLH3FXYE43DWD425,NaN,1.633917,1.642950,1.505346e+12,FAILED,EXPORT_IMAGE,1505345793744
2,0.010833,1505345694593,reducedmeanrunoff_month_Y1960Y2014M11V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,HVHHTVMQI3UI2XE3RMRHJNGB,NaN,1.637317,1.648150,1.505346e+12,FAILED,EXPORT_IMAGE,1505345793482
3,0.011683,1505345693894,reducedmeanrunoff_month_Y1960Y2014M10V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,FNQO727UO4AQSVO7RN4WH56F,NaN,1.615800,1.627483,1.505346e+12,FAILED,EXPORT_IMAGE,1505345791543
4,0.008967,1505345693322,reducedmeanrunoff_month_Y1960Y2014M9V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,UMY3D57LPMRX34SFXAQYPZUU,NaN,1.613967,1.622933,1.505346e+12,FAILED,EXPORT_IMAGE,1505345790698
5,0.010817,1505345692801,reducedmeanrunoff_month_Y1960Y2014M8V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,LNFZ7KOLWIHIVFQGDVYZSDKZ,NaN,1.616900,1.627717,1.505346e+12,FAILED,EXPORT_IMAGE,1505345790464
6,0.009533,1505345692244,reducedmeanrunoff_month_Y1960Y2014M7V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,MIKDWNADAFEUFKVQWOHSMD5E,NaN,1.583133,1.592667,1.505346e+12,FAILED,EXPORT_IMAGE,1505345787804
7,0.012167,1505345691727,reducedmeanrunoff_month_Y1960Y2014M6V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,73LMLHJQYYL6EBCVGH3PE2MV,NaN,1.585550,1.597717,1.505346e+12,FAILED,EXPORT_IMAGE,1505345787590
8,0.009083,1505345691205,reducedmeanrunoff_month_Y1960Y2014M5V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,EQVT3HE4EKRYDKPJQL35BB3P,NaN,1.544417,1.553500,1.505346e+12,FAILED,EXPORT_IMAGE,1505345784415
9,0.007367,1505345690630,reducedmeanrunoff_month_Y1960Y2014M4V20,Cannot overwrite asset 'projects/WRI-Aquaduct/...,Y5VVWP74K5AOIR4UCHOI2YMB,NaN,1.514633,1.522000,1.505346e+12,FAILED,EXPORT_IMAGE,1505345781950


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)